In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import shapely
import matplotlib.pyplot as plt

In [3]:
#JOIN CRASH DATA W TAXI DATA

In [ ]:
#2018

In [4]:
df = pd.read_pickle("../preprocessed_data/2018cleaned.pkl")

In [5]:
crash_df = pd.read_pickle("../preprocessed_data/crashes_w_zones.pkl")

In [6]:
#create date column to join w crash data on
df["date"] = df.pickup_time.dt.date
df['date'] = df['date'].astype('datetime64[ns]')

In [7]:
crash_2018 = crash_df[crash_df.YEAR == 2018]

In [8]:
per_day_per_zone_crash = crash_2018.groupby(["CRASH DATE", "zone"]).size()

In [9]:
per_day_per_zone_crash  = pd.DataFrame(per_day_per_zone_crash) \
.reset_index().rename({0:"num_crashes"}, axis = 1)

In [12]:
df_w_PU_crash = pd.merge(df, per_day_per_zone_crash, how = 'left', \
                      left_on = ['date', 'PULocationID'], right_on = ['CRASH DATE', 'zone'])

In [13]:
df_w_crash = pd.merge(df_w_PU_crash, per_day_per_zone_crash, how = 'left', \
                      left_on = ['date', 'DOLocationID'], right_on = ['CRASH DATE', 'zone'])

In [14]:
df_w_crash = df_w_crash.rename({"num_crashes_x":"PU_crashes", "num_crashes_y":"DO_crashes"}, axis = 1)

In [15]:
df_w_crash = df_w_crash.drop(['CRASH DATE_x', 'zone_x','CRASH DATE_y', 'zone_y'], axis = 1)

In [18]:
df_w_crash['PU_crashes'] = df_w_crash['PU_crashes'].fillna(0)

In [19]:
df_w_crash['DO_crashes'] = df_w_crash['DO_crashes'].fillna(0)

In [2]:
df_w_crash = pd.read_pickle("../preprocessed_data/2018taxi_w_crash.pkl")

In [ ]:
#2019

In [4]:
df19 = pd.read_pickle("../preprocessed_data/2019cleaned.pkl")

In [6]:
#create date column to join w crash data on
df19["date"] = df19.pickup_time.dt.date
df19['date'] = df19['date'].astype('datetime64[ns]')

In [7]:
crash_2019 = crash_df[crash_df.YEAR == 2019]

In [8]:
per_day_per_zone_crash = crash_2019.groupby(["CRASH DATE", "zone"]).size()

In [9]:
per_day_per_zone_crash  = pd.DataFrame(per_day_per_zone_crash) \
.reset_index().rename({0:"num_crashes"}, axis = 1)

In [12]:
df_w_PU_crash = pd.merge(df19, per_day_per_zone_crash, how = 'left', \
                      left_on = ['date', 'PULocationID'], right_on = ['CRASH DATE', 'zone'])

In [13]:
df_w_crash = pd.merge(df_w_PU_crash, per_day_per_zone_crash, how = 'left', \
                      left_on = ['date', 'DOLocationID'], right_on = ['CRASH DATE', 'zone'])

In [14]:
df_w_crash = df_w_crash.rename({"num_crashes_x":"PU_crashes", "num_crashes_y":"DO_crashes"}, axis = 1)

In [15]:
df_w_crash = df_w_crash.drop(['CRASH DATE_x', 'zone_x','CRASH DATE_y', 'zone_y'], axis = 1)

In [18]:
df_w_crash['PU_crashes'] = df_w_crash['PU_crashes'].fillna(0)

In [19]:
df_w_crash['DO_crashes'] = df_w_crash['DO_crashes'].fillna(0)

In [2]:
df_w_crash = pd.read_pickle("../preprocessed_data/2019taxi_w_crash.pkl")

In [ ]:
#ADD ZONE CENTROIDS TO ESTIMATE DISTANCE

In [21]:
zones = fiona.open("../preprocessed_data/zones/fixed_taxi_zones.shp")

In [2]:
zone_lookup  = pd.read_csv("../raw_data/zone_lookup.csv")

In [6]:
zone_lookup = zone_lookup.drop([263, 264])

In [3]:
centres = []
for i in zones:
    shape = shapely.geometry.asShape(i['geometry'])
    zone_id = i['properties']['LocationID']
    centre = (shape.centroid.x, shape.centroid.y)
    centres.append(centre)
    

NameError: name 'zones' is not defined

In [24]:
centres = pd.Series(centres)

In [25]:
zone_lookup['centre'] = centres

In [ ]:
zone_lookup = zone_lookup.drop('')

In [ ]:
#2018

In [ ]:
df_w_crash = pd.read_pickle('../preprocessed_data/2018taxi_w_crash.pkl')

In [28]:
df_w_PU_centres = pd.merge(df_w_crash, zone_lookup, how = 'left', \
                      left_on = ['PULocationID'], right_on = ['LocationID'])

In [30]:
df_w_centres = pd.merge(df_w_PU_centres, zone_lookup, how = 'left', \
                      left_on = ['DOLocationID'], right_on = ['LocationID'])

In [33]:
df_w_centres = df_w_centres.rename({'centre_x': 'PUcentre', 'centre_y': 'DOcentre', \
                                    'Borough_x':'PUBorough', 'Borough_y':'DOBorough'}, axis = 1)

In [35]:
df_w_centres = df_w_centres.drop(['Unnamed: 0_x', 'LocationID_x', 'Zone_x',
       'service_zone_x', 'Unnamed: 0_y', 'LocationID_y', 'Zone_y', 'service_zone_y'], axis = 1)

In [37]:
import geopy.distance

In [38]:
def approx_dist(x, y):
    if x == y:
        return 0
    return geopy.distance.distance(x, y).mi

In [ ]:
#Takes a long time (3+ hours)
df_w_centres['approx_dist'] = np.vectorize(approx_dist)(df_w_centres["PUcentre"], df_w_centres["DOcentre"])

In [ ]:
df_w_centres['approx_dist'] = df_w_centres['approx_dist'].replace(0,0.5)

In [41]:
df_w_centres.to_pickle('../preprocessed_data/2018_w_dists.pkl')

In [ ]:
#2019

In [ ]:
df_w_crash = pd.read_pickle('../preprocessed_data/2019taxi_w_crash.pkl')

In [28]:
df_w_PU_centres = pd.merge(df_w_crash, zone_lookup, how = 'left', \
                      left_on = ['PULocationID'], right_on = ['LocationID'])

In [30]:
df_w_centres = pd.merge(df_w_PU_centres, zone_lookup, how = 'left', \
                      left_on = ['DOLocationID'], right_on = ['LocationID'])

In [33]:
df_w_centres = df_w_centres.rename({'centre_x': 'PUcentre', 'centre_y': 'DOcentre', \
                                    'Borough_x':'PUBorough', 'Borough_y':'DOBorough'}, axis = 1)

In [35]:
df_w_centres = df_w_centres.drop(['Unnamed: 0_x', 'LocationID_x', 'Zone_x',
       'service_zone_x', 'Unnamed: 0_y', 'LocationID_y', 'Zone_y', 'service_zone_y'], axis = 1)

In [ ]:
#Takes a long time (3+ hours)
df_w_centres['approx_dist'] = np.vectorize(approx_dist)(df_w_centres["PUcentre"], df_w_centres["DOcentre"])

In [ ]:
df_w_centres['trip_distance'].describe(percentiles = [0.05])

In [ ]:
df_w_centres['approx_dist'] = df_w_centres['approx_dist'].replace(0,0.5)

In [41]:
df_w_centres.to_pickle('../preprocessed_data/2019_w_dists.pkl')

In [ ]:
#2018

In [3]:
df_w_crash = pd.read_pickle("../preprocessed_data/2018_w_dists.pkl")

In [ ]:
#JOINING WEATHER DATA

In [4]:
weather = pd.read_csv("../preprocessed_data/important_daily_weather.csv")

In [6]:
weather['just_date'] = weather['just_date'].astype("datetime64[ns]")

In [7]:
all_data = pd.merge(df_w_crash, weather, how = 'left', \
                      left_on = ['date'], right_on = ['just_date'])

In [8]:
all_data = all_data.drop(['Unnamed: 0', 'just_date'], axis = 1)

In [10]:
all_data.to_pickle('../preprocessed_data/2018_all_data.pkl')

In [ ]:
#2019

In [3]:
df_w_crash = pd.read_pickle("../preprocessed_data/2019_w_dists.pkl")

In [ ]:
#JOINING WEATHER DATA

In [4]:
weather = pd.read_csv("../preprocessed_data/important_daily_weather.csv")

In [6]:
weather['just_date'] = weather['just_date'].astype("datetime64[ns]")

In [7]:
all_data = pd.merge(df_w_crash, weather, how = 'left', \
                      left_on = ['date'], right_on = ['just_date'])

In [8]:
all_data = all_data.drop(['Unnamed: 0', 'just_date'], axis = 1)

In [10]:
all_data.to_pickle('../preprocessed_data/2019_all_data.pkl')